In [2]:
%matplotlib inline

GridSearch with Census Data
===========================


このノートブックでは、Fairlearn と Fairness ダッシュボードを使用して、Census データセットの予測値を生成する方法を示しています。このデータセットは分類問題で、32,000人の個人に関するデータが与えられた場合、その個人の年収が5万ドル以上か以下かを予測します。

このノートでは、これをローン決定問題として扱うことにします。ラベルは、各個人が過去にローンを返済したかどうかを示していることにします。このデータを使って、以前見たことのない個人がローンを返済するかどうかを予測する予測器を訓練します。モデルの予測は、個人にローンを提供すべきかどうかを決定するために使われることを前提としています。

まず、公平性を意識していない予測器を学習し、それが「デモグラフィック・パリティ」と呼ばれる特定の公平性の概念の下で、不公平な決定をもたらすことを示します。次に、Fairlearnパッケージの`GridSearch`{.sourceCode}アルゴリズムを適用することで、不公平感を軽減します。

> This notebook shows how to use Fairlearn and the Fairness dashboard to generate predictors for the Census dataset. This dataset is a classification problem - given a range of data about 32,000 individuals, predict whether their annual income is above or below fifty thousand
> dollars per year.
>
> For the purposes of this notebook, we shall treat this as a loan decision problem. We will pretend that the label indicates whether or not each individual repaid a loan in the past. We will use the data to train a predictor to predict whether previously unseen individuals will repay a loan or not. The assumption is that the model predictions are used to decide whether an individual should be offered a loan.
>
> We will first train a fairness-unaware predictor and show that it leads to unfair decisions under a specific notion of fairness called *demographic parity*. We then mitigate unfairness by applying the `GridSearch`{.sourceCode} algorithm from the Fairlearn package.

Load and preprocess the data set
================================

fairlearn.datasetsのfetch_adult関数を使って、データセットをダウンロードします。まず、使用する様々なモジュールをインポートします。

`FairlearnDashboard`{.sourceCode}はFairlearnの一部として開発されなくなりました。ウィジェット自体は[the raiwidgets package](https://pypi.org/project/raiwidgets/)に移動しました。Fairlearnでは、既存の機能の一部を`matplotlib`{.sourceCode}ベースのビジュアライゼーションで提供します。

> We download the data set using fetch\_adult function in fairlearn.datasets. We start by importing the various modules we're going to use:
>
> > **note**
> >
> > The `FairlearnDashboard`{.sourceCode} is no longer being developed as part of Fairlearn. The widget itself has been moved to [the raiwidgets package](https://pypi.org/project/raiwidgets/). Fairlearn will provide some of the existing functionality through `matplotlib`{.sourceCode}-based visualizations.

In [2]:
from sklearn.model_selection import train_test_split
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity, ErrorRate

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [4]:
from raiwidgets import FairnessDashboard
FairlearnDashboard = FairnessDashboard

fairlearn.datasetsを使って、データの読み込みと検査を行うことができます。

> We can now load and inspect the data by using the fairlearn.datasets module:

In [5]:
from sklearn.datasets import fetch_openml
data = fetch_openml(data_id=1590, as_frame=True)
X_raw = data.data
Y = (data.target == '>50K') * 1
X_raw

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States
4,18.0,NaN,103497.0,Some-college,10.0,Never-married,NaN,Own-child,White,Female,0.0,0.0,30.0,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27.0,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States
48838,40.0,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States
48839,58.0,Private,151910.0,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,Female,0.0,0.0,40.0,United-States
48840,22.0,Private,201490.0,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,Male,0.0,0.0,20.0,United-States


ここでは、各個人の性別をセンシティブな特徴（0が女性、1が男性を表す）として扱い、この特別なケースでは、この特徴を分離してメインデータから削除します。次に、標準的なデータの前処理を行い、MLアルゴリズムに適した形式に変換します。

> We are going to treat the sex of each individual as a sensitive feature (where 0 indicates female and 1 indicates male), and in this particular case we are going separate this feature out and drop it from the main data. We then perform some standard data preprocessing steps to convert the data into a format suitable for the ML algorithms


In [7]:
A = X_raw["sex"]
X = X_raw.drop(labels=['sex'], axis=1)
X = pd.get_dummies(X)

sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

le = LabelEncoder()
Y = le.fit_transform(Y)

最後に、データをトレーニングセットとテストセットに分けます。

> Finally, we split the data into training and test sets:


In [8]:
X_train, X_test, Y_train, Y_test, A_train, A_test = train_test_split(X_scaled,
                                                                     Y,
                                                                     A,
                                                                     test_size=0.2,
                                                                     random_state=0,
                                                                     stratify=Y)

# Work around indexing bug
X_train = X_train.reset_index(drop=True)
A_train = A_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
A_test = A_test.reset_index(drop=True)

Training a fairness-unaware predictor
=====================================

Fairlearnの効果を示すために、まず公平性を取り入れていない標準的なML予測器を訓練します。デモのスピードを上げるために、シンプルなsklearn.linear\_model.LogisticRegressionクラスを使用します。

> To show the effect of Fairlearn we will first train a standard ML predictor that does not incorporate fairness. For speed of demonstration, we use the simple sklearn.linear\_model.LogisticRegression class:

In [9]:
unmitigated_predictor = LogisticRegression(solver='liblinear', fit_intercept=True)

unmitigated_predictor.fit(X_train, Y_train)

LogisticRegression(solver='liblinear')

この予測値をFairnessダッシュボードに読み込み、その公平性を評価することができます。

> We can load this predictor into the Fairness dashboard, and assess its fairness:


In [9]:
FairlearnDashboard(sensitive_features=A_test,
                  #  sensitive_feature_names=['sex'],
                   y_true=Y_test,
                   y_pred={"unmitigated": unmitigated_predictor.predict(X_test)})

Fairness started at http://localhost:5000


正確さの格差を見ると、男性は女性の約3倍の誤差があることがわかります。さらに興味深いのは、機会の格差で、男性は女性の3倍の割合でローンを提供されています。

訓練データからこの特徴を削除したにもかかわらず、予測器は依然として性別によって差別されています。これは、予測器をフィットさせる際に敏感な特徴を無視するだけでは、不公平感が解消されないことを示しています。一般的に、除去された特徴と相関のある他の特徴が十分にあり、差別的な影響をもたらすことになります。

> Looking at the disparity in accuracy, we see that males have an error about three times greater than the females. More interesting is the disparity in opportunity - males are offered loans at three times the rate of females.
> 
> Despite the fact that we removed the feature from the training data, our predictor still discriminates based on sex. This demonstrates that simply ignoring a sensitive feature when fitting a predictor rarely eliminates unfairness. There will generally be enough other features correlated with the removed feature to lead to disparate impact.

Mitigation with GridSearch
==========================

fairlearn.reducations.GridSearchクラスは、[Agarwal et al.2018](https://arxiv.org/abs/1803.02453)のexponentiated gradient reductionの簡易版を実装しています。ユーザーは標準的なML推定量を供給しますが、これはブラックボックスとして扱われます。GridSearchは、再ラベル化と再重み付けのシーケンスを生成することで動作し、それぞれについて予測器を訓練します。

この例では、公平性の指標として、（性別という敏感な特徴を持つ）人口学的平準化を指定します。人口統計学的平準化とは、個人が敏感なクラスのメンバーシップとは無関係に機会を提供される（この例ではローンが承認される）ことを必要とします（すなわち、女性と男性が同じ割合でローンを提供されるべきです）。ここでは単純化のためにこの指標を使用していますが、一般的には適切な公平性の指標は明らかではありません。

> The fairlearn.reductions.GridSearch class implements a simplified version of the exponentiated gradient reduction of [Agarwal et al.  2018](https://arxiv.org/abs/1803.02453). The user supplies a standard ML estimator, which is treated as a blackbox. GridSearch works by generating a sequence of relabellings and reweightings, and trains a predictor for each.
>
> For this example, we specify demographic parity (on the sensitive feature of sex) as the fairness metric. Demographic parity requires that individuals are offered the opportunity (are approved for a loan in this example) independent of membership in the sensitive class (i.e., females and males should be offered loans at the same rate). We are using this metric for the sake of simplicity; in general, the appropriate fairness metric will not be obvious.


In [3]:
sweep = GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                   constraints=DemographicParity(),
                   grid_size=71)

我々のアルゴリズムは，`fit()`{.sourceCode}と`predict()`{.sourceCode}メソッドを提供しており，Pythonの他のMLパッケージと同様の動作をします．ただし，`fit()`{.sourceCode}には2つの追加の引数を指定する必要があります．すなわち，敏感な特徴ラベルの列と，今回のスイープで生成する予測子の数です．

fit()`{.sourceCode}が完了した後， fairlearn.reducations.GridSearchオブジェクトから予測子のフルセットを抽出します．

> Our algorithms provide `fit()`{.sourceCode} and `predict()`{.sourceCode} methods, so they behave in a similar manner to other ML packages in Python. We do however have to specify two extra arguments to `fit()`{.sourceCode} - the column of sensitive feature labels, and also the number of predictors to generate in our sweep.
>
> After `fit()`{.sourceCode} completes, we extract the full set of predictors from the fairlearn.reductions.GridSearch object.


In [10]:
sweep.fit(X_train, Y_train,
          sensitive_features=A_train)

predictors = sweep.predictors_

KeyboardInterrupt: 

これらの予測因子を今すぐFairnessダッシュボードにロードすることができます。しかし、プロットは、その数のためにやや混乱するでしょう。このケースでは、誤差-視差空間で他のものに支配されている予測子を掃引から取り除きます（視差は敏感な特徴についてのみ計算されることに注意してください；他の潜在的に敏感な特徴は軽減されません）。一般的には，（与えられた敏感な特徴の）誤差と視差の厳密な最適化を超えた他の考慮事項があるので，これをしたくないかもしれません。

> We could load these predictors into the Fairness dashboard now. However, the plot would be somewhat confusing due to their number. In this case, we are going to remove the predictors which are dominated in the error-disparity space by others from the sweep (note that the disparity will only be calculated for the sensitive feature; other potentially sensitive features will not be mitigated). In general, one might not want to do this, since there may be other considerations beyond the strict optimization of error and disparity (of the given sensitive feature).


In [ ]:
errors, disparities = [], []
for m in predictors:
    def classifier(X): return m.predict(X)

    error = ErrorRate()
    error.load_data(X_train, pd.Series(Y_train), sensitive_features=A_train)
    disparity = DemographicParity()
    disparity.load_data(X_train, pd.Series(Y_train), sensitive_features=A_train)

    errors.append(error.gamma(classifier)[0])
    disparities.append(disparity.gamma(classifier).max())

all_results = pd.DataFrame({"predictor": predictors, "error": errors, "disparity": disparities})

non_dominated = []
for row in all_results.itertuples():
    errors_for_lower_or_eq_disparity = all_results["error"][all_results["disparity"] <= row.disparity]
    if row.error <= errors_for_lower_or_eq_disparity.min():
        non_dominated.append(row.predictor)

最後に、支配的なモデルを「Fairness」ダッシュボードに入れて、緩和されていないモデルと一緒に表示することができます。

> Finally, we can put the dominant models into the Fairness dashboard, along with the unmitigated model.


In [ ]:
dashboard_predicted = {"unmitigated": unmitigated_predictor.predict(X_test)}
for i in range(len(non_dominated)):
    key = "dominant_model_{0}".format(i)
    value = non_dominated[i].predict(X_test)
    dashboard_predicted[key] = value


FairlearnDashboard(sensitive_features=A_test, sensitive_feature_names=['sex'],
                   y_true=Y_test,
                   y_pred=dashboard_predicted)

パレートフロントが形成されているのがわかります - 予測における精度と格差の間の最適なトレードオフを表す予測子のセットです。理想的なケースでは、(1,0)に予測値があります - 完全に正確で、（センシティブな特徴である "性 "に関して）人口学的なパリティのもとで不公平感がありません。パレートフロントは、我々のデータと推定量の選択に基づいて、この理想に最も近づくことができることを表しています。軸の範囲に注意してください - 視差軸は精度よりも多くの値をカバーしているので、精度の損失が小さくても視差を大幅に減らすことができます。

プロット上の個々のモデルをクリックすると、視差と精度の測定基準をより詳細に調べることができます。実際の例では、関連するビジネス上の制約を考慮して、精度と視差の間の最良のトレードオフを示すモデルを選ぶことになるでしょう。

> We see a Pareto front forming - the set of predictors which represent optimal tradeoffs between accuracy and disparity in predictions. In the ideal case, we would have a predictor at (1,0) - perfectly accurate and without any unfairness under demographic parity (with respect to the sensitive feature "sex"). The Pareto front represents the closest we can come to this ideal based on our data and choice of estimator. Note the range of the axes - the disparity axis covers more values than the accuracy, so we can reduce disparity substantially for a small loss in accuracy.
>
> By clicking on individual models on the plot, we can inspect their metrics for disparity and accuracy in greater detail. In a real example, we would then pick the model which represented the best trade-off between accuracy and disparity given the relevant business constraints.


## 参考になるサイト

- [機械学習の公平性 (プレビュー) - Azure Machine Learning | Microsoft Docs](https://docs.microsoft.com/ja-jp/azure/machine-learning/concept-fairness-ml "機械学習の公平性 (プレビュー) - Azure Machine Learning | Microsoft Docs")
- [Azure Machine Learning Studioで使って理解を深めるResponsible AI (Part 2) – Fairlearn&lt;公平性&gt;編 – #Azure #AzureMachineLearning | DevelopersIO](https://dev.classmethod.jp/articles/mrmo-azure-ai-ml-fairlearn-20210408/ "Azure Machine Learning Studioで使って理解を深めるResponsible AI (Part 2) – Fairlearn&lt;公平性&gt;編 – #Azure #AzureMachineLearning | DevelopersIO")